In [1]:
import pyspark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1618853444290_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
from pyspark.sql import functions as F
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import lower, col
from collections import Counter
import pandas as pd
import re
sqlContext = SQLContext(sc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
input_bucket = 's3://final-project-spark-twitter-date'
path_favorites = '/data/favorites.csv'
path_friends = '/data/Friends.csv'
path_retweets = '/data/Retweets.csv'
path_statuses = '/data/statuses_colon.csv'
path_users = '/data/users_colon.csv'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# reading in the 5 twitter tables
df_favorites = sqlContext.read.format('csv').option("inferSchema", "true").option('header','true').load(input_bucket + path_favorites)
df_friends = sqlContext.read.format('csv').option("inferSchema", "true").option('header','true').load(input_bucket + path_friends)
df_retweets = sqlContext.read.format('csv').option("inferSchema", "true").option('header','true').load(input_bucket + path_retweets)
df_statuses = sqlContext.read.format('csv')\
                    .option('delimiter', ';')\
                    .option("quote", "\"")\
                    .option("escape", "\"")\
                    .option("inferSchema", "true")\
                    .option('header','true')\
                    .option('wholeText', 'true')\
                    .option('multiLine', 'true')\
                    .load(input_bucket + path_statuses)
df_users = sqlContext.read.format('csv') \
                    .option('delimiter', ";")\
                    .option('header', "true")\
                    .option("multiLine", "true")\
                    .option('wholeText', 'true')\
                    .option("inferSchema", "true")\
                    .load(input_bucket + path_users)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Answering analytical questions 

#### Q1: Number of statuses by SpaceX account. Answer: 5102

In [6]:
df_users.filter(F.col("screen_name").isin("SpaceX")).collect()[0]['statuses_count']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'5102'

There are 5102 statuses for "SpaceX" Twitter user

#### Q2: Which users tweet the most? Answer: screen_name is 'morinaga_angel'

In [11]:
# cast the statuses count from string to integer type
df_users_cast = df_users.withColumn("statuses_count", df_users["statuses_count"].cast(IntegerType()))

# These are the columns that we will filter for 
cols = ['screen_name', 'statuses_count']

# for when statuses_count are not null, find the the user with the most number of tweets
df_users_cast_filtered = df_users_cast.select(*cols, F.col("statuses_count")\
                                  .cast("int").isNotNull().alias("Value"))\
                                  .filter(F.col("Value")==True)\
                                  .orderBy("statuses_count", ascending=False)\
                                  .limit(1).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------------+-----+
|   screen_name|statuses_count|Value|
+--------------+--------------+-----+
|morinaga_angel|       4351485| true|
+--------------+--------------+-----+

Twitter user "morinaga_angel" has the most tweets in the twitter dataset. As shown above, the name of the user has been substitued with "?" because the characters are in Chinese and are not available in the utf-8 encoding. 

#### Q3: Display the top 5 users by the highest retweets by status ratio.

With this question we will demonstrate that we can convert spark dataframes to tables, and perform SQL queries

In [69]:
# register spark dataframes as tables that can be used to perform SQL queries with
df_statuses.registerTempTable('statuses')
df_users.registerTempTable('users')
 
# sql query that finds the ratio of twitter users with highest number of retweets to number of tweets
sqlContext.sql(
    'SELECT u.user_id, first(u.screen_name) as screen_name, '
    'count(s.user_id) as NumofTweets, '
    'sum(s.retweet_count) as NumofRetweets, '
    'sum(s.retweet_count)/count(s.user_id) as Total '
    'FROM statuses s '
    'INNER JOIN users u '
    'ON u.user_id=s.user_id '
    'GROUP BY 1 '
    'ORDER BY Total DESC '
    'LIMIT 5'
).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+---------------+-----------+-------------+---------+
|            user_id|    screen_name|NumofTweets|NumofRetweets|    Total|
+-------------------+---------------+-----------+-------------+---------+
| 718495181914316801|chadwickboseman|          1|      2169628|2169628.0|
|         2511327758|       Levance_|          1|       563780| 563780.0|
|1003842651567816711|       CH1ONESO|          1|       553333| 553333.0|
|1249456677931208704|  thejulianbass|          1|       542135| 542135.0|
|         2344207064|_JohnnyMontanez|          1|       471821| 471821.0|
+-------------------+---------------+-----------+-------------+---------+

#### Q4: How many statuses mention "SpaceX"? 

In [70]:
df_statuses.select('text').filter("lower(text) like '%spacex%'").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2885

#### (Extra) Q5: What are the number of mentions per twitter user excluding any retweets? 

In [67]:
def get_all_mentions(df):
    ''' get all mentions from df_statuses text field'''
    search_string = r'\b(?<=@)\w+'
    text_with_mentions = df.filter(df['text'].rlike(search_string)).select('text')
    text_with_mentions_list = [row['text'] for row in text_with_mentions.collect()]
    return [mention.split('@')[1] for mention in ' '.join(text_with_mentions_list).split() if mention[0]=='@']

# get all mentions from the "statuses" table
all_mentions = get_all_mentions(df_statuses)

# convert list of mentions to value counts 
all_mentions_counts = Counter(all_mentions)

# convert the collections.Counter dictionary to a pandas dataframe
all_mentions_counts_df = pd.DataFrame.from_dict(all_mentions_counts, orient='index', columns=['mention_count']).reset_index()
all_mentions_counts_df.rename(columns={'index':'screen_name'}, inplace=True)

# converting from pandas to spark df so we can join them together
all_mentions_counts_sp = spark.createDataFrame(all_mentions_counts_df)

# inner merge on the df_users table to only get the count for users that are in our dataset
user_mentions = df_users.join(all_mentions_counts_sp, on='screen_name', how='inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

These are the top 5 Twitter users in our dataset that are mentioned the most in twitter statuses.

In [71]:
user_mentions.select('screen_name', 'mention_count').orderBy('mention_count', ascending=False).limit(5).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-------------+
|  screen_name|mention_count|
+-------------+-------------+
|     elonmusk|         1635|
|         NASA|         1427|
|       SpaceX|         1376|
|Space_Station|         1304|
|        Tesla|         1215|
+-------------+-------------+

There are 2885 statuses that mention the word "spacex". We ensured to lowercase the text and find the word "spacex" so we were as conclusive as possible. 